## Example of building df_possible (all possible player_team combos)

Note: actual df_possible building was done with build_df_possibles.py so it could be run easily on UNIX cluster - this is just to illustrate and test 

In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import time

year = 2017

df_actual = pd.read_csv('pruned_data/pruned_data_{}.csv'.format(year))

df_actual.head()

,player_id,mp,fg,fga,fg3,fg3a,ft,fta,orb,trb,...,tov,pf,pts,player_name,team_id,opp_id,team_pts,opp_pts,boxscore_id,is_playoffs
0,devyn-marble-1,29.0,5.0,16.0,1.0,9.0,6.0,6.0,0.0,5.0,...,0.0,6.0,17.0,Devyn Marble,aris,aries-trikala,75,65,2016-10-08-aris,False
1,spiros-mourtos-1,28.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,...,2.0,1.0,2.0,Spyros Mourtos,aris,aries-trikala,75,65,2016-10-08-aris,False
2,will-cummings-1,21.0,4.0,7.0,1.0,2.0,6.0,6.0,0.0,1.0,...,4.0,5.0,15.0,Will Cummings,aris,aries-trikala,75,65,2016-10-08-aris,False
3,eric-buckner-1,20.0,0.0,2.0,0.0,0.0,3.0,6.0,1.0,8.0,...,1.0,4.0,3.0,Eric Buckner,aris,aries-trikala,75,65,2016-10-08-aris,False
4,michalis-tsairelis-1,20.0,3.0,5.0,0.0,1.0,2.0,2.0,0.0,4.0,...,2.0,1.0,8.0,Michalis Tsairelis,aris,aries-trikala,75,65,2016-10-08-aris,False


In [2]:
# reformat so triangle generator can differentiate easily
df_actual['defense'] = 'Defense_' + df_actual['opp_id']

#change this so its the same as toy example
df_actual.columns = ['player' if x=='player_id' else x for x in df_actual.columns]

df_actual.head()

,player,mp,fg,fga,fg3,fg3a,ft,fta,orb,trb,...,pf,pts,player_name,team_id,opp_id,team_pts,opp_pts,boxscore_id,is_playoffs,defense
0,devyn-marble-1,29.0,5.0,16.0,1.0,9.0,6.0,6.0,0.0,5.0,...,6.0,17.0,Devyn Marble,aris,aries-trikala,75,65,2016-10-08-aris,False,Defense_aries-trikala
1,spiros-mourtos-1,28.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,...,1.0,2.0,Spyros Mourtos,aris,aries-trikala,75,65,2016-10-08-aris,False,Defense_aries-trikala
2,will-cummings-1,21.0,4.0,7.0,1.0,2.0,6.0,6.0,0.0,1.0,...,5.0,15.0,Will Cummings,aris,aries-trikala,75,65,2016-10-08-aris,False,Defense_aries-trikala
3,eric-buckner-1,20.0,0.0,2.0,0.0,0.0,3.0,6.0,1.0,8.0,...,4.0,3.0,Eric Buckner,aris,aries-trikala,75,65,2016-10-08-aris,False,Defense_aries-trikala
4,michalis-tsairelis-1,20.0,3.0,5.0,0.0,1.0,2.0,2.0,0.0,4.0,...,1.0,8.0,Michalis Tsairelis,aris,aries-trikala,75,65,2016-10-08-aris,False,Defense_aries-trikala


#### add all per_min

In [3]:
df_actual.columns

Index(['player', 'mp', 'fg', 'fga', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'trb',
       'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'player_name', 'team_id',
       'opp_id', 'team_pts', 'opp_pts', 'boxscore_id', 'is_playoffs',
       'defense'],
      dtype='object')

In [4]:
stats = ['pts', 'trb', 'ast', 'blk', 'stl']

for stat in stats:
    new_col = stat + '_per_min'
    df_actual[new_col] = df_actual.apply(lambda row: 0 if row['mp'] == 0 else row[stat] / row['mp'], axis=1)

df_actual.head()

,player,mp,fg,fga,fg3,fg3a,ft,fta,orb,trb,...,team_pts,opp_pts,boxscore_id,is_playoffs,defense,pts_per_min,trb_per_min,ast_per_min,blk_per_min,stl_per_min
0,devyn-marble-1,29.0,5.0,16.0,1.0,9.0,6.0,6.0,0.0,5.0,...,75,65,2016-10-08-aris,False,Defense_aries-trikala,0.586207,0.172414,0.000000,0.00,0.034483
1,spiros-mourtos-1,28.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,...,75,65,2016-10-08-aris,False,Defense_aries-trikala,0.071429,0.107143,0.107143,0.00,0.035714
2,will-cummings-1,21.0,4.0,7.0,1.0,2.0,6.0,6.0,0.0,1.0,...,75,65,2016-10-08-aris,False,Defense_aries-trikala,0.714286,0.047619,0.238095,0.00,0.000000
3,eric-buckner-1,20.0,0.0,2.0,0.0,0.0,3.0,6.0,1.0,8.0,...,75,65,2016-10-08-aris,False,Defense_aries-trikala,0.150000,0.400000,0.000000,0.05,0.000000
4,michalis-tsairelis-1,20.0,3.0,5.0,0.0,1.0,2.0,2.0,0.0,4.0,...,75,65,2016-10-08-aris,False,Defense_aries-trikala,0.400000,0.200000,0.050000,0.00,0.050000


### Build df_possible

In [5]:
len(df_actual['player'].unique())

1378

In [6]:
df_actual['player'].unique()

array(['devyn-marble-1', 'spiros-mourtos-1', 'will-cummings-1', ...,
       'okben-ulubay-1', 'louis-olinde-1', 'rodions-kurucs-1'], dtype=object)

In [7]:
len(df_actual['defense'].unique())

89

In [8]:
df_actual['defense'].unique()

array(['Defense_aries-trikala', 'Defense_aris', 'Defense_paok',
       'Defense_doxa-bc', 'Defense_olympiakos', 'Defense_koroivos',
       'Defense_kymis-bc', 'Defense_lavrio', 'Defense_rethymno',
       'Defense_rhodes', 'Defense_aek-athens', 'Defense_apollon-patras',
       'Defense_promitheas', 'Defense_panathinaikos',
       'Defense_fuenlabrada', 'Defense_alba-berlin', 'Defense_cedevita',
       'Defense_gran-canaria', 'Defense_vilnius', 'Defense_khimki',
       'Defense_mzt-skopje', 'Defense_lietkabelis', 'Defense_buducnost',
       'Defense_murcia', 'Defense_kuban', 'Defense_olimpija-ljubljana',
       'Defense_bayern-muenchen', 'Defense_triumph-moscow', 'Defense_ulm',
       'Defense_valencia', 'Defense_novgorod', 'Defense_bilbao',
       'Defense_unicaja-malaga', 'Defense_hapoel-jerusalem',
       'Defense_real-madrid', 'Defense_sevilla', 'Defense_estudiantes',
       'Defense_vitoria', 'Defense_obradoiro', 'Defense_canarias',
       'Defense_barcelona', 'Defense_manresa', 'De

In [9]:
defense_list = []
player_list = []

for defense in df_actual['defense'].unique():
    #print(defense)
    for player in df_actual['player'].unique():
        defense_list.append(defense)
        player_list.append(player)
        
data_dict = {'defense' : defense_list, 'player' : player_list}        

df_possible = pd.DataFrame(data=data_dict)

In [10]:
len(df_possible)

122642

In [11]:
df_possible.head()

,defense,player
0,Defense_aries-trikala,devyn-marble-1
1,Defense_aries-trikala,spiros-mourtos-1
2,Defense_aries-trikala,will-cummings-1
3,Defense_aries-trikala,eric-buckner-1
4,Defense_aries-trikala,michalis-tsairelis-1


### real results from df_actual -> df_possible

In [12]:
# test filtering

df_test_luka = df_actual[(df_actual['player'] == 'luka-doncic-1') & (df_actual['defense'] == 'Defense_barcelona')]

df_test_luka

,player,mp,fg,fga,fg3,fg3a,ft,fta,orb,trb,...,team_pts,opp_pts,boxscore_id,is_playoffs,defense,pts_per_min,trb_per_min,ast_per_min,blk_per_min,stl_per_min
9058,luka-doncic-1,13.0,1.0,3.0,0.0,1.0,0.0,0.0,0.0,2.0,...,75,85,2016-11-06-barcelona,False,Defense_barcelona,0.153846,0.153846,0.000000,0.0,0.000000
12000,luka-doncic-1,17.0,1.0,4.0,1.0,3.0,1.0,2.0,3.0,8.0,...,76,75,2017-03-12-real-madrid,False,Defense_barcelona,0.235294,0.470588,0.176471,0.0,0.058824
26343,luka-doncic-1,15.0,3.0,4.0,1.0,2.0,1.0,2.0,1.0,3.0,...,102,63,2016-11-18-barcelona,False,Defense_barcelona,0.533333,0.200000,0.266667,0.0,0.066667
29587,luka-doncic-1,21.0,2.0,5.0,0.0,2.0,1.0,2.0,2.0,6.0,...,85,69,2017-03-22-real-madrid,False,Defense_barcelona,0.238095,0.285714,0.190476,0.0,0.142857


In [13]:
# count (games played)

len(df_test_luka)

4

In [14]:
# average all games

stats_per_min = ['pts_per_min', 'trb_per_min', 'ast_per_min', 'blk_per_min', 'stl_per_min']

for stat in stats_per_min:
    print(stat)
    print(df_test_luka[stat].mean())


pts_per_min
0.29014221073044605
trb_per_min
0.2775371687136393
ast_per_min
0.1584033613445378
blk_per_min
0.0
stl_per_min
0.06708683473389356


### Apply to all rows in df_possible

In [15]:
# apply above tests to all rows

stats_per_min = ['pts_per_min', 'trb_per_min', 'ast_per_min', 'blk_per_min', 'stl_per_min']

for index, row in df_possible.iterrows():
    if index % 1000 == 0:
        print('index: {}'.format(index))
        print(time.time())
    
    df_filtered = df_actual[(df_actual['player'] == row['player']) & (df_actual['defense'] == row['defense'])] 
    
    df_possible.loc[index, 'times_played'] = len(df_filtered)
    
    if len(df_filtered) == 0:
        for stat in stats_per_min:
            df_possible.loc[index, stat] = 0.0
    else:
        for stat in stats_per_min:
            df_possible.loc[index, stat] = df_filtered[stat].mean()



index: 0
1522347356.9580796
index: 1000
1522347369.0761092
index: 2000
1522347380.5472918
index: 3000
1522347391.4709466
index: 4000
1522347402.446885
index: 5000
1522347414.2045438
index: 6000
1522347426.0520928
index: 7000
1522347439.4012582
index: 8000
1522347451.8541803
index: 9000
1522347463.519383
index: 10000
1522347475.520958
index: 11000
1522347487.2840903
index: 12000
1522347498.8458142
index: 13000
1522347510.5987582
index: 14000
1522347522.3547027
index: 15000
1522347534.2577622
index: 16000
1522347546.4461286
index: 17000
1522347558.6571908
index: 18000
1522347570.6505713
index: 19000
1522347581.3064156
index: 20000
1522347592.3831913
index: 21000
1522347603.8075967
index: 22000
1522347615.3862908
index: 23000
1522347627.7650232
index: 24000
1522347640.0756774
index: 25000
1522347650.8721082
index: 26000
1522347662.03856
index: 27000
1522347673.3298602
index: 28000
1522347684.7090616
index: 29000
1522347696.1948988
index: 30000
1522347707.623399
index: 31000
1522347719.107

In [16]:
df_possible.head()

,defense,player,times_played,pts_per_min,trb_per_min,ast_per_min,blk_per_min,stl_per_min
0,Defense_aries-trikala,devyn-marble-1,1.0,0.586207,0.172414,0.000000,0.00,0.034483
1,Defense_aries-trikala,spiros-mourtos-1,2.0,0.160714,0.178571,0.053571,0.00,0.017857
2,Defense_aries-trikala,will-cummings-1,2.0,0.460591,0.075534,0.170772,0.00,0.000000
3,Defense_aries-trikala,eric-buckner-1,1.0,0.150000,0.400000,0.000000,0.05,0.000000
4,Defense_aries-trikala,michalis-tsairelis-1,2.0,0.283333,0.141667,0.025000,0.00,0.025000
